In [1]:
from Graph import *
import time

## Vhodni podatki

Vhodne podatke preberemo iz datoteke, ki smo jo naredili s pomočjo nauty traces in ukaza geng -c n.
V repozitoriju se trenutno nahajajo primeri za grafe velikosti 5, 8 in 9.

In [2]:
G = read_graph6("graph_examples/graphs_8.txt")


## Reševanje problema

Reševanja problema se lotimo na 2 načina. Prvi bo poizkušal minimizirati število konfliktov v grafu v polinomskem času. Konflikt je taka povezava $(u,v) \in E(G)$ za katero velja $\omega(u) = \omega(v)$. Drugi način bo vzel *neoznačeno* povezavo $e$ in problem razdelil na $3$ podprobleme, kjer bo v vsakem nastavil utež na tej povezavi na 1, 2 ali 3. 

Graf predstavimo z nekaj dodatnimi podatkovnimi strukturami, ki nam omogočajo laže reševanje konfliktov. To so:
* `edge_weights` : slovar, ki vsaki povezavi dodeli neko utež
* `node_sums` : slovar, ki vsakemu vozlišču dodeli njegovo utež, torej vsoto uteži na incidenčnih povezavah
* `sums` : slovar, ki vsaki  možni vsoti uteži dodeli seznam vozlišč, ki imajo to utež.
* `conflicts` : Množica konfliktinih povezav.
* `history` : Slovar, ki povezavi dodeli neko *označeno* vrednost in vrsti red dodelitve. Označene povezave kasneje ne morejo biti spremenjene.

Na začetku izvajanja nastavimo grafu naključne uteži kar nam omogoča `randomize_weights` funkcija.


### Minimizacija konfliktov:

Naj bo $c = (u, v)$ konflikt. Edini način, da rešimo konflikt $c$ in ohranimo obstoječo utežitev na povezavi $(u,v)$ je s spremembo utežitve na eni izmed naslednjih povezav:
$$E_{c} = (\{u\} \times N(u)) \cup (\{v\} \times N(v)) \setminus \{(u, v)\}$$

Za vsako povezavo $e \in E_c$ spremenimo utež na tej povezavi. Zraven seveda popravimo celotno strukturo grafa, ter zabeležimo število kofliktov pred in po spremembi utežitve. Obravnavamo povezavo, ki najbolj zmanjša skupno ševilo konfliktov. V kolokor je po tej spremembi skupno število strogo manjšo kot pred spremembo, to spremembo tudi izvedemo. Drugače vrnemo false in grafa ne rešujemo naprej.

Možna izboljšava: Trenutno utež popravim na neko vrednost, ki je različna ob obstoječe. Smiselno bi bilo probati obe veljavni spremembi in si zabeležiti rezultate pri obeh.


### Branching (+ local search)

V tem primeru želimo v najslabšem primeru preveriti vse grafe in vrniti false, v kolikor ne obstaja utežitev za ta graf (tak rezultat seveda ni pričakovan in bi pomenil, da domneva ne velja. Vendar trenutno je morda kje še kakšna napaka in bi tak rezultat bil bolj povod za iskanje napake).
Tega se lotimo na naslednji način:
* Nastavimo naključne uteži
* Minimiziramo konflikte z prejšnjim algoritmom.
* V kolikor utežitev ni veljavna, vzamemo povezavo, ki je še nismo označili ter problem razdelimo na 3 podprobleme ter jih rekurzivno rešimo. Vsak podproblem torej zopet najprej minimiziramo oz. poizkušamo rešit z zgornjim algoritmom (minimiziranje morda ni najboljša beseda, saj minimalno število konfliktov še ne pomeni nujno tega, da smo blizu rešitve). 
* Pri reševanju podproblemov z zgornjim algoritmom smo posebaj pozorni na reševanje posameznega konflikta. Naj bo $H$ množica *označenih* povezav, ki smo jih nastavili pred rekurzivnimi klici. V koliko obstaja konflikt $c$ za katerega je $E_c \cap H$ prazna množica, potem ta konflikt ni rešljiv z dosedanjo označitvijo povezav zato lahko to vejo izvajanja zaključimo. To storimo z klicem `UnsolvableConflictException`.



### Rezultati za n = 8

#### Local search

V spodnji celici je napisana zanka, ki poizkuša vsak graf iz seznama G rešiti s pomočjo *local search* algoritma.

In [6]:
l = None
print(len(G))
not_solvable = []
start_time = time.time()
for i in range(len(G)):
    g = Graph(G[i])
    g.randomize_weights()
    if len(g.conflicts) != 0:
        succ = g.solve()
        if succ == False:
            not_solvable.append(i)
            l=g
           
print(len(not_solvable))
print('It takes {0} seconds to solve it!'.format(str( time.time() - start_time)))

11117
1251
It takes 3.4026620388031006 seconds to solve it!


* **Vseh grafov**: 11117
* **Neuspešno rešenih**: 1251
* **Čas**: 3.4s

#### Branching (+ local search)
Sedaj rešimo še isti nabor grafov, kjer uporabljamo *branching* na uteži na povezavah in v najslabšem primeru pregledamo vse možne utežitve.

In [7]:
l = None
print(len(G))
not_solvable = []
start_time = time.time()
for i in range(len(G)):
    g = Graph(G[i])
    g.randomize_weights()
    if len(g.conflicts) != 0:
        succ = solve_recursive(g)
        if succ == False:
            not_solvable.append(i)
            l=g
           
print(len(not_solvable))
print('It takes {0} seconds to solve it!'.format(str( time.time() - start_time)))

11117
0
It takes 3.6536648273468018 seconds to solve it!


* **Vseh grafov**: 11117
* **Neuspešno rešenih**: 0
* **Čas**: 3.6s

## Rezultati (za n=9)

#### Local search

In [9]:
G = read_graph6("graph_examples/graphs_9.txt")

In [10]:
l = None
print(len(G))
not_solvable = []
start_time = time.time()
for i in range(len(G)):
    g = Graph(G[i])
    g.randomize_weights()
    if len(g.conflicts) != 0:
        succ = g.solve()
        if succ == False:
            not_solvable.append(i)
            l=g
           
print(len(not_solvable))
print('It takes {0} seconds to solve it!'.format(str( time.time() - start_time)))

261080
26694
It takes 100.60344529151917 seconds to solve it!


* **Vseh grafov**: 261080
* **Neuspešno rešenih**: 26694
* **Čas**: 100.1s

#### Branching (+ local search)

In [11]:
l = None
print(len(G))
not_solvable = []
start_time = time.time()
for i in range(len(G)):
    g = Graph(G[i])
    g.randomize_weights()
    if len(g.conflicts) != 0:
        succ = solve_recursive(g)
        if succ == False:
            not_solvable.append(i)
            l=g
           
print(len(not_solvable))
print('It takes {0} seconds to solve it!'.format(str( time.time() - start_time)))

261080
0
It takes 108.85072922706604 seconds to solve it!


* **Vseh grafov**: 261080
* **Neuspešno rešenih**: 0
* **Čas**: 108.1s